In [1]:
pip install mysql-connector-python

  Obtaining dependency information for mysql-connector-python from https://files.pythonhosted.org/packages/d3/b8/6798a0f91e595c0704a94c1e32a00ca930f77cb8f0f96c7e4dad4f9db1fe/mysql_connector_python-8.1.0-cp311-cp311-win_amd64.whl.metadata
     ---------------------------------------- 0.0/527.0 kB ? eta -:--:--
     --------- ---------------------------- 133.1/527.0 kB 2.6 MB/s eta 0:00:01
     ------------------------------- ------ 440.3/527.0 kB 4.6 MB/s eta 0:00:01
     -------------------------------------- 527.0/527.0 kB 4.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/10.8 MB ? eta -:--:--
   - -------------------------------------- 0.3/10.8 MB 17.3 MB/s eta 0:00:01
   -- ------------------------------------- 0.6/10.8 MB 12.0 MB/s eta 0:00:01
   --- ------------------------------------ 0.9/10.8 MB 7.1 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.8 MB 6.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.0/10.8 MB 5.7 MB/s eta 

In [4]:
import mysql.connector
from mysql.connector import Error
import pandas as pd

In [12]:
def create_server_connection(host_name, user_name, user_password):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

In [13]:
connection = create_server_connection("192.168.0.142", "root", "some_pass")

MySQL Database connection successful


In [16]:
#Função Conectando ao Banco de dados
def create_db_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name, 
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")

    return connection

#Função para execução de consultas
def execute_query(connection, query):
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query successful")
    except Error as err:
        print(f"Error: '{err}'")

In [17]:
#Lendo dados
def read_query(connection, query):
    cursor = connection.cursor()
    result = None
    try:
        cursor.execute(query)
        result = cursor.fetchall()
        return result
    except Error as err:
        print(f"Error: '{err}'")

q1 = """
SELECT *
FROM rio;
"""

connection = create_db_connection("192.168.0.142", "root", "some_pass", "teste")
results = read_query(connection, q1)

for result in results:
    print(result)        

MySQL Database connection successful
(101, 'Guandú', 'Fictício...', 2)
(102, 'Mambucaba', 'teste', 1)
(103, 'Frade', 'teste', 1)
(104, 'Parati-mirim', 'teste', 1)
(105, 'Centro ', 'teste', 1)
(106, 'Jacareí', 'teste', 1)
(107, 'Bracui', 'teste', 1)
(108, 'Taquari', 'teste', 1)
(109, 'Jacuecanga', 'teste', 1)
(110, 'Japuiba', 'teste', 1)
(111, 'São Roque', 'teste', 1)


In [18]:
# Retorna uma lista de listas e cria um DataFrame do Pandas
from_db = []

for result in results:
    result = list(result)
    from_db.append(result)


columns = ["rios_id", "nome", "lat/long", "projeto"]
df = pd.DataFrame(from_db, columns=columns)

In [19]:
df

,rios_id,nome,lat/long,projeto
0,101,Guandú,Fictício...,2
1,102,Mambucaba,teste,1
2,103,Frade,teste,1
3,104,Parati-mirim,teste,1
4,105,Centro,teste,1
5,106,Jacareí,teste,1
6,107,Bracui,teste,1
7,108,Taquari,teste,1
8,109,Jacuecanga,teste,1
9,110,Japuiba,teste,1
